# Taarifa Data Set. Problem Understanding

The point of this project is to predict which pumps that help to clean and maintain water well's throughout Tanzania are to be fixed before they are broken. My immediate response to this problem is to look at the data, see what is missing, what is present. My next step is to look at domain knowledge. I am specifically going to look at the types of water pumps used in these wells. I feel that it is important to see if there have been any recalls on any of the models used. After cleaning the datasets I will create a baseline model using Gaussian Naive Bayes. From here I will tune by using hyper parameter optimisation. I feel that the second model I will use will be stacking. 

There are two methods of looking at the data. One through downloading the API and it's appropriate software. The second are the csv files that are given on the competition website with Data Camp. 
I have decided to go through the second route. I have attempted to download the API but have run into some problems which I will later explore but cannot do so given the time frame. 

I have not conducted a ternary classification, which is what we are dealing with here in the Taarifa dataset. 

# Data Understanding
https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/

In [3]:
#import!

import numpy as np
import pandas as pd

In [4]:
submission_data = pd.read_csv('/Users/aqureshi/Desktop/pumpitup/SubmissionFormat.csv')
submission_data.head()

,id,status_group
0,50785,predicted label
1,51630,predicted label
2,17168,predicted label
3,45559,predicted label
4,49871,predicted label


In [7]:
test_set_data = pd.read_csv('/Users/aqureshi/Desktop/pumpitup/testset_values.csv')
test_set_data.tail()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,never pay,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,annually,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,never pay,soft,good,insufficient,insufficient,dam,dam,surface,communal standpipe,communal standpipe
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,never pay,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe
14849,68707,0.0,2013-02-13,Government Of Tanzania,481,Government,34.765054,-11.226012,Kwa Mzee Chagala,0,...,never pay,soft,good,dry,dry,spring,spring,groundwater,communal standpipe,communal standpipe


## It may be good to look at water quality, quantity, source, source_type, source_class, waterpoint_type, waterpoint_type_group, location. 
I think these are good started with the waterwells. I will do some research on whether these play a role with the type of waterpumps needed due to location, and cause of breakdown. Also if the different funders use the same suppliers or different ones. 

In [9]:
training_set_data = pd.read_csv('/Users/aqureshi/Desktop/pumpitup/training_set_values.csv')
training_set_data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


## We will be working with the train_set_data throughout and will finish off by running the test_set_data once!

In [13]:
# checking the shape
print(training_set_data.shape)

(59400, 40)


In [14]:
print(test_set_data.shape)

(14850, 40)


In [15]:
#This is a 80/20 split

In [16]:
training_set_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [19]:
# dropping null values
training_set_data.dropna().head()


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe
13,50495,0.0,2013-03-15,Lawatefuka Water Supply,1368,Lawatefuka water sup,37.092574,-3.181783,Kwa John Izack Mmari,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
14,53752,0.0,2012-10-20,Biore,0,WEDECO,34.364073,-3.629333,Mwabasabi,0,...,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump


In [20]:
#found a new dataset which is refined
wpms = pd.read_csv('/Users/aqureshi/Desktop/DS_021720/TaarifaAPI/wpms_refine.csv')
wpms.head()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,gid,objectid,valid_from,valid_to,amount_tsh,breakdown_year,date_recorded,funder,gps_height,installer,...,source_group,source_class,status,status_group,wp_type,wp_type_group,region_code,district_code,district_name,ward_code
0,52387944,52365558,2013-07-08T00:00:00Z,2101-10-01T00:00:00Z,0.0,0.0,2013-03-16T00:00:00Z,NaN,1393,NaN,...,borehole,groundwater,functional,functional,communal standpipe,communal standpipe,2,3,Arusha Urban,12
1,52387945,52365560,2013-07-08T00:00:00Z,2101-10-01T00:00:00Z,50.0,0.0,2013-03-13T00:00:00Z,NaN,1410,NaN,...,borehole,groundwater,functional,functional,communal standpipe,communal standpipe,2,3,Arusha Urban,22
2,52387946,52365562,2013-07-08T00:00:00Z,2101-10-01T00:00:00Z,50.0,0.0,2013-03-13T00:00:00Z,NaN,1432,NaN,...,borehole,groundwater,functional,functional,communal standpipe,communal standpipe,2,6,Arusha,73
3,52387947,52365564,2013-07-08T00:00:00Z,2101-10-01T00:00:00Z,100.0,2008.0,2013-03-13T00:00:00Z,NaN,1429,NaN,...,borehole,groundwater,functional,functional,communal standpipe,communal standpipe,2,6,Arusha,73
4,52387948,52365566,2013-07-08T00:00:00Z,2101-10-01T00:00:00Z,100.0,0.0,2013-03-13T00:00:00Z,NaN,1426,NaN,...,borehole,groundwater,functional needs repair,functional needs repair,communal standpipe,communal standpipe,2,6,Arusha,83


In [21]:
print(wpms.shape)

(74673, 58)


In [24]:
wpms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74673 entries, 0 to 74672
Data columns (total 58 columns):
gid                       74673 non-null int64
objectid                  74673 non-null int64
valid_from                74673 non-null object
valid_to                  74673 non-null object
amount_tsh                74629 non-null float64
breakdown_year            73452 non-null float64
date_recorded             74673 non-null object
funder                    70151 non-null object
gps_height                74673 non-null int64
installer                 70121 non-null object
longitude                 74673 non-null float64
latitude                  74673 non-null float64
wptname                   74673 non-null object
num_privcon               74591 non-null float64
basin                     74673 non-null object
village                   74673 non-null object
subvillage                74200 non-null object
village_reg_num           19181 non-null object
region_name              

In [22]:
#This dataset seems to not have a split between training and test. 
#I also see that funder and installer have been left out. 
#Here I think it's best to stick to the above